<a href="https://colab.research.google.com/github/oaalikouche12-bot/sql-northwind-practice/blob/main/Copia_de_UPF_MUDAB_2025_Caso_Pr%C3%A1ctico_1_SQL_OMAR_AALIKOUCHE_ALICHANE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el entorno

## Instalación de PostgreSQL e importación de librerías

### Creación del servidor PostgreSQL
El siguiente código levanta un PostgreSQL en la máquina del google COLAB y lo inicializa, tanto el usuario como la contraseña para acceder a la base de datos se pueden cambiar pero por defecto serán "postgres" ambas.

El siguiente código también genera una base de datos llamada "northwind" que será la que usaremos.

`psql` es el comando del servicio de postgresql, es el que usamos para mandarle comandos a la hora de editar el usuario y crear la base de datos.


In [ ]:
# Importo librería para ejecutar comandos de shell
import subprocess

# Instalamos postgresql server en la máquina del COLAB
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Ejecutar el comando para verificar el estado del servicio PostgreSQL
result = subprocess.run(['service', 'postgresql', 'status'], stdout=subprocess.PIPE)
STATUS_OUTPUT = result.stdout.decode('utf-8')
print(STATUS_OUTPUT)

# Verificar si la palabra "online" está presente en la salida del comando y que muestre online.
if "online" in STATUS_OUTPUT:
    print("PostgreSQL se encuentra online :). Se ha levantado la BBDD.")
else:
    print("PostgreSQL no se encuentra online")

# Variables para la configuración (nombre base de datos, usuario, puerto, ...)
SQL_PRACTICE_DATABASE_NAME="northwind"
SQL_PRACTICE_DATABASE_HOST="localhost"
SQL_PRACTICE_DATABASE_PORT=5432
SQL_PRACTICE_DATABASE_USER="postgres"
SQL_PRACTICE_DATABASE_PASS="postgres"

# Dropeamos BBDD porque no le gusta matar sesiones. (Daba error al ejecutar el Collab porque había otra sesión) + Cambio del password para el username `postgres`
!sudo -u postgres psql -U postgres -c "DROP DATABASE {SQL_PRACTICE_DATABASE_NAME} WITH (FORCE);"
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD '{SQL_PRACTICE_DATABASE_PASS}';"

# Creación de la base de datos
!sudo -u postgres psql -U postgres -c "DROP DATABASE IF EXISTS {SQL_PRACTICE_DATABASE_NAME};"
!sudo -u postgres psql -U postgres -c "CREATE DATABASE {SQL_PRACTICE_DATABASE_NAME};"

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
 * Starting PostgreSQL 14 database server
   ...done.
14/main (port 5432): online

PostgreSQL se encuentra online :). Se ha levantado la BBDD.
DROP DATABASE
ALTER ROLE
NOTICE:  database "northwind" does not exist, skipping
DROP DATABASE
CREATE DATABASE


### Importamos las librerías necesarias
- **[psycopg2](https://www.psycopg.org/)**. Librería que nos permite interactuar con servidores y clientes de postgresql
- **pandas**. Librearía conocida para el análisis y transformación de datos.

In [ ]:
import psycopg2
import pandas as pd

## Creación de tablas y carga de datos en el servidor PostgreSQL

Como hemos visto, la [base de datos de Northwind](https://dbdocs.io/akweiwonder3/Northwind-Database?schema=public&view=relationships&table=order_details) tiene varias tablas, el objetivo de esta práctica es familiarizarse con los comandos de consulta de SQL y no de creación de base de datos, por lo que usaremos un script que nos creará todas las tablas y nos cargará todos los datos.

### Creación y configuración base de datos

In [ ]:
# Primero creamos la conexión usando la librearía psycopg2
# https://pypi.org/project/psycopg2/
connection = psycopg2.connect(host=SQL_PRACTICE_DATABASE_HOST,
                              database=SQL_PRACTICE_DATABASE_NAME,
                              user=SQL_PRACTICE_DATABASE_USER,
                              password=SQL_PRACTICE_DATABASE_PASS)
connection.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT);

# Creamos el cursor en la conexión, el cursor es nuestra "puerta de acceso" a la base de datos usando
# la conexión configurada
cursor = connection.cursor()

In [ ]:
# Creación de tablas
%env SQL_PRACTICE_DATABASE_NAME = SQL_PRACTICE_DATABASE_NAME
%env SQL_PRACTICE_DATABASE_HOST = SQL_PRACTICE_DATABASE_HOST
%env SQL_PRACTICE_DATABASE_PORT = SQL_PRACTICE_DATABASE_PORT
%env SQL_PRACTICE_DATABASE_USER = SQL_PRACTICE_DATABASE_USER
%env SQL_PRACTICE_DATABASE_PASS = SQL_PRACTICE_DATABASE_PASS

!PGPASSWORD=$SQL_PRACTICE_DATABASE_PASS psql -q -h $SQL_PRACTICE_DATABASE_HOST -p $SQL_PRACTICE_DATABASE_PORT -U $SQL_PRACTICE_DATABASE_USER -d $SQL_PRACTICE_DATABASE_NAME -f northwind_ddl_full.sql

!PGPASSWORD=$SQL_PRACTICE_DATABASE_PASS psql -q -h $SQL_PRACTICE_DATABASE_HOST -p $SQL_PRACTICE_DATABASE_PORT -U $SQL_PRACTICE_DATABASE_USER -d $SQL_PRACTICE_DATABASE_NAME -f northwind_data.sql

env: SQL_PRACTICE_DATABASE_NAME=SQL_PRACTICE_DATABASE_NAME
env: SQL_PRACTICE_DATABASE_HOST=SQL_PRACTICE_DATABASE_HOST
env: SQL_PRACTICE_DATABASE_PORT=SQL_PRACTICE_DATABASE_PORT
env: SQL_PRACTICE_DATABASE_USER=SQL_PRACTICE_DATABASE_USER
env: SQL_PRACTICE_DATABASE_PASS=SQL_PRACTICE_DATABASE_PASS


# Ejercicios





In [ ]:
# Método de pandas para lanzar consultas SQL y cargarlas en un dataframe
pd.read_sql("select * from categories", connection)

/tmp/ipython-input-96391461.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("select * from categories", connection)


,category_id,category_name,description,picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",[]
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",[]
2,3,Confections,"Desserts, candies, and sweet breads",[]
3,4,Dairy Products,Cheeses,[]
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",[]
5,6,Meat/Poultry,Prepared meats,[]
6,7,Produce,Dried fruit and bean curd,[]
7,8,Seafood,Seaweed and fish,[]


In [ ]:
# En lugar de usar cursor.execute(sql) y cursor.fetchall() para mostrar resultados
# mejor usar la función read_sql de Pandas que nos devuelve directamente un dataframe

# Creamos una función más simple para no tener que escribir todo cada vez que lancemos una query
def query(sql):
  return pd.read_sql(sql, connection)

## 1. Informe completo de todos los pedidos (orders) desde el origen de los tiempos
Es decir, sacar todos los Orders tal cual, con toda la información

In [ ]:
query("""select * from orders""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,Mexico
826,11074,SIMOB,7,1998-05-06,1998-06-03,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,1998-05-06,1998-06-03,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Switzerland
828,11076,BONAP,4,1998-05-06,1998-06-03,None,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France


## 2. Se necesitan dos reports sobre los Customers:
- Lista de Customers/Contacts que son managers
- Lista de Customer/Contacts que no son managers

Ambos reportes tendrán que estar ordenados alfabéticamente por país y nombre para cargarlos en el CRM (formato específico), además las columnas tienen que tener los siguientes nombres para evitar problemas en la carga: compañia, nombre, posicion, direccion, ciudad, pais.

_Funciones útiles: [SQL Like](https://www.w3schools.com/sql/sql_like.asp), [SQL IN](https://www.w3schools.com/sql/sql_in.asp), [SQL Aliases](https://www.w3schools.com/sql/sql_alias.asp), [DISTINCT](https://www.w3schools.com/sql/sql_distinct.asp)_

In [ ]:
query("""select company_name as Compañia, contact_name as nombre, contact_title as posicion, address as direccion, city as ciudad, country as pais
 from customers where contact_title like '%Manager' order by country, contact_name""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,compañia,nombre,posicion,direccion,ciudad,pais
0,Piccolo und mehr,Georg Pipps,Sales Manager,Geislweg 14,Salzburg,Austria
1,Ernst Handel,Roland Mendel,Sales Manager,Kirchgasse 6,Graz,Austria
2,Suprêmes délices,Pascale Cartrain,Accounting Manager,"Boulevard Tirou, 255",Charleroi,Belgium
3,Que Delícia,Bernardo Batista,Accounting Manager,"Rua da Panificadora, 12",Rio de Janeiro,Brazil
4,Hanari Carnes,Mario Pontes,Accounting Manager,"Rua do Paço, 67",Rio de Janeiro,Brazil
5,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,Brazil
6,Bottom-Dollar Markets,Elizabeth Lincoln,Accounting Manager,23 Tsawassen Blvd.,Tsawassen,Canada
7,Vaffeljernet,Palle Ibsen,Sales Manager,Smagsloget 45,Århus,Denmark
8,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,Finland
9,La maison d'Asie,Annette Roulet,Sales Manager,1 rue Alsace-Lorraine,Toulouse,France


In [ ]:
query("""select company_name as Compañia, contact_name as nombre, contact_title as posicion, address as direccion, city as ciudad, country as pais
 from customers where contact_title not like '%Manager' order by country, contact_name""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,compañia,nombre,posicion,direccion,ciudad,pais
0,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,Argentina
1,Rancho grande,Sergio Gutiérrez,Sales Representative,Av. del Libertador 900,Buenos Aires,Argentina
2,Océano Atlántico Ltda.,Yvonne Moncada,Sales Agent,Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,Argentina
3,Maison Dewey,Catherine Dewey,Sales Agent,Rue Joseph-Bens 532,Bruxelles,Belgium
4,Tradição Hipermercados,Anabela Domingues,Sales Representative,"Av. Inês de Castro, 414",Sao Paulo,Brazil
5,Gourmet Lanchonetes,André Fonseca,Sales Associate,"Av. Brasil, 442",Campinas,Brazil
6,Familia Arquibaldo,Aria Cruz,Marketing Assistant,"Rua Orós, 92",Sao Paulo,Brazil
7,Ricardo Adocicados,Janete Limeira,Assistant Sales Agent,"Av. Copacabana, 267",Rio de Janeiro,Brazil
8,Queen Cozinha,Lúcia Carvalho,Marketing Assistant,"Alameda dos Canàrios, 891",Sao Paulo,Brazil
9,Comércio Mineiro,Pedro Afonso,Sales Associate,"Av. dos Lusíadas, 23",Sao Paulo,Brazil


## 3. Se necesitan 4 reportes distintos:
1. Lista de Orders de 1996
2. Lista de Employees, con su nombre completo en un único campo, su edad y país
3. Número de Orders hechos desde el 15 de Julio de 1997
4. Media de días que se tarda en enviar un pedido desde que se hace

_Funciones útiles: [CONCAT](https://www.postgresqltutorial.com/postgresql-string-functions/postgresql-concat-function/), [DATE_PART](https://www.postgresql.org/docs/8.1/functions-datetime.html), [COUNT](https://www.w3schools.com/sql/sql_count.asp), [AVG](https://www.w3schools.com/sql/sql_avg.asp)_

In [ ]:
query("""select * from orders where order_date between '1996-01-01' and '1996-12-31'""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,10395,HILAA,6,1996-12-26,1997-01-23,1997-01-03,1,184.41,HILARION-Abastos,Carrera 22 con Ave. Carlos Soublette #8-35,San Cristóbal,Táchira,5022,Venezuela
148,10396,FRANK,1,1996-12-27,1997-01-10,1997-01-06,3,135.35,Frankenversand,Berliner Platz 43,München,None,80805,Germany
149,10397,PRINI,5,1996-12-27,1997-01-24,1997-01-02,1,60.26,Princesa Isabel Vinhos,Estrada da saúde n. 58,Lisboa,None,1756,Portugal
150,10398,SAVEA,2,1996-12-30,1997-01-27,1997-01-09,3,89.16,Save-a-lot Markets,187 Suffolk Ln.,Boise,ID,83720,USA


In [ ]:
query("""select first_name || ' ' || last_name as nombre_completo, date_part('year', age(birth_date)) as edad, country
from employees""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,nombre_completo,edad,country
0,Andrew Fuller,73.0,USA
1,Janet Leverling,62.0,USA
2,Margaret Peacock,88.0,USA
3,Steven Buchanan,70.0,UK
4,Michael Suyama,62.0,UK
5,Robert King,65.0,UK
6,Laura Callahan,67.0,USA
7,Anne Dodsworth,59.0,UK
8,Nancy Davolio,76.0,USA


In [ ]:
query("""select count(order_id) from orders where order_date >= '1997-07-15'""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,count
0,479


In [ ]:
query("""select avg(shipped_date - order_date) from orders
where shipped_date is not null""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,avg
0,8.491965


## 4. Reportes para preparar la temporada de Navidad
1. Lista todos los pedidos, saca un campo extra _is_christmas_season_ que sea True si el pedido se hizo en temporada de navidad o False si se hizo fuera de temporada
2. Necesitamos hacer un Sanity Check en la base de datos para comprobar que está todo en orden antes de empezar la temporada de Navidad... ¿Podrías comprobar que el número de pedidos en la tabla **orders** coincide con el número de pedidos de la tabla **order_details**?
3. Queremos enviar un regalo a empleados y clientes. Extrae en un mismo reporte (query) la lista de ambos, los campos que se necesitan son el nombre completo, la dirección, el codigo postal, ciudad y país

Consideramos temporada de Navidad desde el 1 de Diciembre al 31 de Diciembre de cualquier año.

_Funciones útiles: [CASE WHEN](https://www.w3schools.com/sql/sql_case.asp), [DATE_PART](https://www.postgresql.org/docs/8.1/functions-datetime.html), [UNION](https://www.w3schools.com/sql/sql_union.asp), [DISTINCT](https://www.w3schools.com/sql/sql_distinct.asp)_

In [ ]:
query("""
select
    order_id,
    order_date,
    case
        when date_part('month', order_date) = 12
             and date_part('day', order_date) between 1 and 31
        then true
        else false
    end as is_christmas_season
from orders
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,order_date,is_christmas_season
0,10248,1996-07-04,False
1,10249,1996-07-05,False
2,10250,1996-07-08,False
3,10251,1996-07-08,False
4,10252,1996-07-09,False
...,...,...,...
825,11073,1998-05-05,False
826,11074,1998-05-06,False
827,11075,1998-05-06,False
828,11076,1998-05-06,False


In [ ]:
query("""
select
    (select count(order_id) from orders) as orders_count,
    (select count(distinct order_id) from order_details) as order_details_count
""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,orders_count,order_details_count
0,830,830


In [ ]:
query("""
select
    company_name as nombre_completo,
    address,
    postal_code,
    city,
    country
from customers

union

select
    first_name || ' ' || last_name as nombre_completo,
    address,
    postal_code,
    city,
    country
from employees
""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,nombre_completo,address,postal_code,city,country
0,Janet Leverling,722 Moss Bay Blvd.,98033,Kirkland,USA
1,Ricardo Adocicados,"Av. Copacabana, 267",02389-890,Rio de Janeiro,Brazil
2,B's Beverages,Fauntleroy Circus,EC2 5NT,London,UK
3,Furia Bacalhau e Frutos do Mar,Jardim das rosas n. 32,1675,Lisboa,Portugal
4,Around the Horn,120 Hanover Sq.,WA1 1DP,London,UK
...,...,...,...,...,...
95,Spécialités du monde,"25, rue Lauriston",75016,Paris,France
96,La maison d'Asie,1 rue Alsace-Lorraine,31000,Toulouse,France
97,Folk och fä HB,Åkergatan 24,S-844 67,Bräcke,Sweden
98,Chop-suey Chinese,Hauptstr. 29,3012,Bern,Switzerland


## 5. Investigar el retraso de algunos Orders 1/2
Podemos considerar un Order como retrasado cuando la fecha de envío es superior a la fecha requerida.
1. ¿Cuántos Orders hay con retraso?
2. Sacar una lista de todos los orders con el nombre de los shippers (distribuidores)
3. Sacar una lista única de los distribuidores que tienen retrasos
4. Sacar una lista única de todos los distribuidores, marcando los que han tenido retraso alguna vez y los que no, sólo para pedidos hechos durante el año 1998
5. Listar la lista de pedidos con sus empleados asociados que se enviarón después de la fecha requerida (required_date)

_Funciones útiles: [JOINS](https://www.w3schools.com/sql/sql_join.asp), [UNION](https://www.w3schools.com/sql/sql_union.asp), [DISTINCT](https://www.w3schools.com/sql/sql_distinct.asp), [WITH](https://www.postgresql.org/docs/current/queries-with.html#QUERIES-WITH-SELECT)_

In [ ]:
query("""
select count(*) as orders_retrasados
from orders
where shipped_date > required_date
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,orders_retrasados
0,37


In [ ]:
query("""
select
    o.order_id,
    s.company_name as shipper_name,
    o.shipped_date,
    o.required_date
from orders o
join shippers s on o.ship_via = s.shipper_id
where o.shipped_date > o.required_date
""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,shipper_name,shipped_date,required_date
0,10264,Federal Shipping,1996-08-23,1996-08-21
1,10271,United Package,1996-08-30,1996-08-29
2,10280,Speedy Express,1996-09-12,1996-09-11
3,10302,United Package,1996-10-09,1996-10-08
4,10309,Speedy Express,1996-10-23,1996-10-17
5,10320,Federal Shipping,1996-10-18,1996-10-17
6,10380,Federal Shipping,1997-01-16,1997-01-09
7,10423,Federal Shipping,1997-02-24,1997-02-06
8,10427,United Package,1997-03-03,1997-02-24
9,10433,Federal Shipping,1997-03-04,1997-03-03


In [ ]:
query("""
select distinct
    s.company_name as shipper_name
from orders o
join shippers s on o.ship_via = s.shipper_id
where o.shipped_date > o.required_date
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,shipper_name
0,United Package
1,Speedy Express
2,Federal Shipping


In [ ]:
query("""
select
    s.company_name as shipper_name,
    case
        when count(case when o.shipped_date > o.required_date then 1 end) > 0
        then 'tuvo retrasos'
        else 'sin retrasos'
    end as estado_retraso
from orders o
join shippers s on o.ship_via = s.shipper_id
where date_part('year', o.order_date) = 1998
group by s.company_name
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,shipper_name,estado_retraso
0,Federal Shipping,Tuvo retrasos
1,Speedy Express,Tuvo retrasos
2,United Package,Tuvo retrasos


In [ ]:
query("""
select
    o.order_id,
    o.shipped_date,
    o.required_date,
    e.first_name || ' ' || e.last_name as empleado
from orders o
join employees e on o.employee_id = e.employee_id
where o.shipped_date > o.required_date
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,shipped_date,required_date,empleado
0,10264,1996-08-23,1996-08-21,Michael Suyama
1,10271,1996-08-30,1996-08-29,Michael Suyama
2,10280,1996-09-12,1996-09-11,Andrew Fuller
3,10302,1996-10-09,1996-10-08,Margaret Peacock
4,10309,1996-10-23,1996-10-17,Janet Leverling
5,10320,1996-10-18,1996-10-17,Steven Buchanan
6,10380,1997-01-16,1997-01-09,Laura Callahan
7,10423,1997-02-24,1997-02-06,Michael Suyama
8,10427,1997-03-03,1997-02-24,Margaret Peacock
9,10433,1997-03-04,1997-03-03,Janet Leverling


## 6. Investigar el retraso de algunos orders 2/2
1. ¿Cuántos pedidos retrasados hay por año?
2. ¿Cuántos pedidos retrasados tiene cada distribuidor? ¿Y por año?
3. Top 2 empleados con más pedidos retrasados
4. Top 2 meses con más pedidos retrasados (mes y año)
5. ¿Hay alguna relación entre el número de productos de un pedido y el retraso?

_Funciones útiles: [GROUP BY](https://www.w3schools.com/sql/sql_groupby.asp), [JOINS](https://www.w3schools.com/sql/sql_join.asp)_

In [ ]:
# ¿Cuántos pedidos retrasados hay por año?
query(""" select
    date_part('year', required_date) as año,
    count(*) as pedidos_retrasados
from orders
where shipped_date > required_date
group by date_part('year', required_date)
order by año
""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,año,pedidos_retrasados
0,1996.0,6
1,1997.0,21
2,1998.0,10


In [ ]:
# ¿Cuántos pedidos retrasados tiene cada distribuidor y por año?
query("""
select
    s.company_name as distribuidor,
    date_part('year', o.required_date) as año,
    count(*) as pedidos_retrasados
from orders o
join shippers s on o.ship_via = s.shipper_id
where o.shipped_date > o.required_date
group by s.company_name, date_part('year', o.required_date)
order by distribuidor, año
""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,distribuidor,año,pedidos_retrasados
0,Federal Shipping,1996.0,2
1,Federal Shipping,1997.0,6
2,Federal Shipping,1998.0,1
3,Speedy Express,1996.0,2
4,Speedy Express,1997.0,5
5,Speedy Express,1998.0,5
6,United Package,1996.0,2
7,United Package,1997.0,10
8,United Package,1998.0,4


In [ ]:
# Top 2 empleados con más pedidos retrasados
query("""
select
    e.first_name || ' ' || e.last_name as empleado,
    count(*) as total_retrasos
from orders o
join employees e on o.employee_id = e.employee_id
where o.shipped_date > o.required_date
group by empleado
order by total_retrasos desc
limit 2
""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,empleado,total_retrasos
0,Margaret Peacock,10
1,Janet Leverling,5


In [ ]:
# Top 2 meses con más pedidos retrasados (mes y año)
query("""
select
    date_part('year', required_date) as año,
    date_part('month', required_date) as mes,
    count(*) as total_retrasos
from orders
where shipped_date > required_date
group by año, mes
order by total_retrasos desc
limit 2
""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,año,mes,total_retrasos
0,1998.0,4.0,4
1,1998.0,1.0,4


In [ ]:
# ¿Hay relación entre el número de productos y el retraso?
query("""
select
    o.order_id,
    count(od.product_id) as cantidad_productos,
    case
        when o.shipped_date > o.required_date then true
        else false
    end as retrasado
from orders o
join order_details od on o.order_id = od.order_id
group by o.order_id, retrasado
order by cantidad_productos desc
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,cantidad_productos,retrasado
0,11077,25,False
1,10847,6,True
2,10657,6,False
3,10979,6,False
4,10893,5,False
...,...,...,...
825,10608,1,False
826,10942,1,False
827,10878,1,False
828,10771,1,False


### 7. Estudio de los empleados
1. ¿Cuál es la distribución de empleados en los distintos territorios?
2. ¿Cuál es la distribución de los empleados en estos rangos de edad 30-40, 40-60, 60+?
3. Reporte que muestre el número de empleados y clientes que hay en cada ciudad que tenga empleados (es decir, no estamos interesados en las ciudades sin empleados)

_Funciones útiles: [CASE WHEN](https://www.w3schools.com/sql/sql_case.asp), [DATE_PART](https://www.postgresql.org/docs/8.1/functions-datetime.html), [UNION](https://www.w3schools.com/sql/sql_union.asp)_

In [ ]:
query("""select
    t.territory_description as territorio,
    count(e.employee_id) as cantidad_empleados
from employees e
join employee_territories et on e.employee_id = et.employee_id
join territories t on et.territory_id = t.territory_id
group by t.territory_description
order by cantidad_empleados desc""")




/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,territorio,cantidad_empleados
0,New York,2
1,Santa Cruz,1
2,Westboro,1
3,Philadelphia,1
4,Phoenix,1
5,San Francisco,1
6,Roseville,1
7,Southfield,1
8,Rockville,1
9,Morristown,1


In [ ]:
query("""select
    case
        when date_part('year', age(birth_date)) between 30 and 40 then '30-40'
        when date_part('year', age(birth_date)) between 41 and 60 then '40-60'
        when date_part('year', age(birth_date)) > 60 then '60+'
        else 'Menor de 30'
    end as rango_edad,
    count(*) as cantidad_empleados
from employees
group by rango_edad
order by rango_edad""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,rango_edad,cantidad_empleados
0,40-60,1
1,60+,8


In [ ]:
query("""with empleados_por_ciudad as (
    select city, count(*) as num_empleados
    from employees
    group by city
),
clientes_por_ciudad as (
    select city, count(*) as num_clientes
    from customers
    group by city
)
select
    e.city,
    e.num_empleados,
    coalesce(c.num_clientes, 0) as num_clientes
from empleados_por_ciudad e
left join clientes_por_ciudad c on e.city = c.city
order by e.num_empleados desc""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,city,num_empleados,num_clientes
0,London,4,6
1,Seattle,2,1
2,Kirkland,1,1
3,Tacoma,1,0
4,Redmond,1,0


## 8. Eficiencia de la operación según la relación empleado - cliente - pedido
1. Teniendo en cuenta los pedidos gestionados por cada empleado, ¿cuántos clientes gestiona de media cada empleado?
2. ¿Cuál es el empleado (nombre) que ha gestionado más clientes? ¿Y más pedidos?
3. Saca una lista con empleados (nombre) que incluya sólo aquellos que han gestionado más de 100 pedidos


In [ ]:
# ¿Cuántos clientes gestiona de media cada empleado?
query("""select round(avg(num_clientes),2) as media_clientes_por_empleado
from (
    select e.employee_id, count(distinct o.customer_id) as num_clientes
    from orders o
    join employees e on o.employee_id = e.employee_id
    group by e.employee_id
) sub""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,media_clientes_por_empleado
0,51.56


In [ ]:
# ¿Cuál es el empleado (nombre) que ha gestionado más clientes? ¿Y más pedidos?
query("""select e.first_name || ' ' || e.last_name as empleado,
count(distinct o.customer_id) as clientes_gestionados
from orders o
join employees e on o.employee_id = e.employee_id
group by e.employee_id, e.first_name, e.last_name
order by clientes_gestionados desc
limit 1""")

query("""select e.first_name || ' ' || e.last_name as empleado,
count(o.order_id) as pedidos_gestionados
from orders o
join employees e on o.employee_id = e.employee_id
group by e.employee_id, e.first_name, e.last_name
order by pedidos_gestionados desc
limit 1""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,empleado,pedidos_gestionados
0,Margaret Peacock,156


In [ ]:

# Lista de empleados (nombre) que han gestionado más de 100 pedidos
query("""select e.first_name || ' ' || e.last_name as empleado,
count(o.order_id) as total_pedidos
from orders o
join employees e on o.employee_id = e.employee_id
group by e.employee_id, e.first_name, e.last_name
having count(o.order_id) > 100
order by total_pedidos desc""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,empleado,total_pedidos
0,Margaret Peacock,156
1,Janet Leverling,127
2,Nancy Davolio,123
3,Laura Callahan,104


## 9. Reporte que incluya un resumen de las siguientes métricas (todo en la misma tabla)
1. Número total de pedidos
2. Número total de pedidos retrasados
3. % de pedidos retrasados
4. Media de pedidos gestionados por empleado

Todas esas métricas deberán de aparecer por año y més (el ejemplo de abajo es orientativo, los números no son reales):

| año     | mes | total_pedidos | pedidos_retrasados | % pedidos retrasados | pedidos_por_empleado
|----------|----------|----------|----------|----------|----------|
| 1996    |  enero  | 134 | 43 |0.03 |19 |
| 1996    | febrero   | 532 | 63 |0.02 |103 |
| 1996    | marzo   | 149 | 34 |0.18 |25 |

In [ ]:
query("""
select
    date_part('year', order_date) as año,
    to_char(order_date, 'month') as mes,
    count(order_id) as total_pedidos,
    count(case when shipped_date > required_date then 1 end) as pedidos_retrasados,
    round(
        100.0 * count(case when shipped_date > required_date then 1 end) / count(*),
        2
    ) as porcentaje_retrasados,
    round(
        avg(num_pedidos_por_empleado),
        2
    ) as pedidos_por_empleado
from orders,
(
    select employee_id, count(order_id) as num_pedidos_por_empleado
    from orders
    group by employee_id
) sub
group by año, mes, date_part('month', order_date)
order by año, date_part('month', order_date)
""")


/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,año,mes,total_pedidos,pedidos_retrasados,porcentaje_retrasados,pedidos_por_empleado
0,1996.0,july,198,9,4.55,92.22
1,1996.0,august,225,18,8.00,92.22
2,1996.0,september,207,18,8.70,92.22
3,1996.0,october,234,9,3.85,92.22
4,1996.0,november,225,0,0.00,92.22
5,1996.0,december,279,9,3.23,92.22
6,1997.0,january,297,18,6.06,92.22
7,1997.0,february,261,18,6.90,92.22
8,1997.0,march,270,9,3.33,92.22
9,1997.0,april,279,9,3.23,92.22


## 10. Comprobaciones finales:
1. ¿Cuáles son las categorías que tienen los productos más caros?
2. ¿Cuáles son los productos (nombre) que tienen unidades en orden y cuáles son sus categorías y distribuidores?
3. ¿Cuál es el precio total de cada pedido?

In [ ]:
# ¿Cuáles son las categorías que tienen los productos más caros?
query("""
select c.category_name, max(p.unit_price) as precio_maximo
from products p
join categories c on p.category_id = c.category_id
group by c.category_name
order by precio_maximo desc
""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,category_name,precio_maximo
0,Beverages,263.50
1,Meat/Poultry,123.79
2,Confections,81.00
3,Seafood,62.50
4,Dairy Products,55.00
5,Produce,53.00
6,Condiments,43.90
7,Grains/Cereals,38.00


In [ ]:
# ¿Cuáles son los productos (nombre) que tienen unidades en orden y cuáles son sus categorías y distribuidores?
query("""
select distinct
    p.product_name,
    c.category_name,
    s.company_name as distribuidor
from products p
join categories c on p.category_id = c.category_id
join suppliers s on p.supplier_id = s.supplier_id
join order_details od on p.product_id = od.product_id
where od.quantity > 0
order by p.product_name
""")

/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,product_name,category_name,distribuidor
0,Alice Mutton,Meat/Poultry,"Pavlova, Ltd."
1,Aniseed Syrup,Condiments,Exotic Liquids
2,Boston Crab Meat,Seafood,New England Seafood Cannery
3,Camembert Pierrot,Dairy Products,Gai pâturage
4,Carnarvon Tigers,Seafood,"Pavlova, Ltd."
...,...,...,...
72,Uncle Bob's Organic Dried Pears,Produce,Grandma Kelly's Homestead
73,Valkoinen suklaa,Confections,Karkki Oy
74,Vegie-spread,Condiments,"Pavlova, Ltd."
75,Wimmers gute Semmelknödel,Grains/Cereals,Plutzer Lebensmittelgroßmärkte AG


In [ ]:
# ¿Cuál es el precio total de cada pedido?
query("""
select
    o.order_id,
    round(sum(od.unit_price * od.quantity * (1 - od.discount))) as precio_total
from orders o
join order_details od on o.order_id = od.order_id
group by o.order_id
order by o.order_id
""")



/tmp/ipython-input-1710432509.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, connection)


,order_id,precio_total
0,10248,440.0
1,10249,1863.0
2,10250,1553.0
3,10251,654.0
4,10252,3598.0
...,...,...
825,11073,300.0
826,11074,232.0
827,11075,498.0
828,11076,793.0


## Extra: Window Functions
Aunque esta sección no se evaluará, se recomienda la realización por parte del alumno que quiera profundizar y mejorar su nivel de SQL.
1. Por cada pedido y producto de Order Details, crear una columna llamada __order_page__ indicando el número de página de cada pedido (es decir, un mismo pedido con 3 productos tendrá un producto con un order page 1, otro con un order page 2 y otro con un order page 3)
2. Sacar la evolución del beneficio total obtenido por los pedidos a lo largo del tiempo (beneficio acumulado)
3. Asignar a cada empleado un ranking numérico según el beneficio generado por la gestión de pedidos, es decir el empleado con la posicion 1 será el que haya tenido más beneficio en ventas
4. Reporte donde se muestre el ID del pedido, el empleado que lo gestionó, la fecha, y el acumulado de pedidos de ese empleado hasta ese momento
5. Crear un reporte donde se muestre la evolución del beneficio por mes y año, además queremos que cada año el acumulado sea independiente, es decir que el 1 de Enero el acumulado se reinicie a 0 (la tabla tendrá tres columnas: año, mes y beneficio)